In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

Primero, se busca calcular la matriz de distancias eucledianas entre zonas y tiendas.

In [4]:
zonas_data = pd.read_csv('zonas_archivo.csv')
tiendas_data = pd.read_csv('tiendas_archivo.csv')

coords_zonas = zonas_data[['x_zona', 'y_zona']].values
coords_tiendas = tiendas_data[['pos_x', 'pos_y']].values

#Se calcula la matriz de distancias entre zonas y tiendas
dist_matrix = cdist(coords_zonas, coords_tiendas, metric='euclidean')

#Se convierte a df para facilitar lectura
dist_df = pd.DataFrame(
    dist_matrix,
    index=zonas_data['id_zona'],
    columns=tiendas_data['id_tienda']
)

print(dist_df.head())

id_tienda         1          2          3          4          5         6   \
id_zona                                                                      
1          24.596748  56.080300  50.566788  63.529521  56.885851  4.472136   
2          23.706539  55.569776  50.159745  63.063460  56.718604  3.605551   
3          22.825424  55.072679  49.769469  62.609903  56.568542  2.828427   
4          21.954498  54.589376  49.396356  62.169124  56.435804  2.236068   
5          21.095023  54.120237  49.040799  61.741396  56.320511  2.000000   

id_tienda         7          8          9          10         11         12  \
id_zona                                                                       
1          30.805844  47.265209  21.954498  56.320511  30.083218  46.861498   
2          30.000000  47.169906  21.095023  56.044625  29.732137  46.227697   
3          29.206164  47.095647  20.248457  55.785303  29.410882  45.607017   
4          28.425341  47.042534  19.416488  55.542776  29.

Luego, la matríz de distancias eucledianas entre solo zonas.

In [5]:
dist_matrix = cdist(coords_zonas, coords_zonas, metric='euclidean')

dist_df = pd.DataFrame(
    dist_matrix,
    index=zonas_data['id_zona'],
    columns=zonas_data['id_zona']
)

print(dist_df.head())

id_zona  1     2     3     4     5     6     7     8     9     10    ...  \
id_zona                                                              ...   
1         0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0  ...   
2         1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0  ...   
3         2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0  ...   
4         3.0   2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0   6.0  ...   
5         4.0   3.0   2.0   1.0   0.0   1.0   2.0   3.0   4.0   5.0  ...   

id_zona       2091       2092       2093       2094       2095       2096  \
id_zona                                                                     
1        71.840100  72.124892  72.422372  72.732386  73.054774  73.389373   
2        71.568149  71.840100  72.124892  72.422372  72.732386  73.054774   
3        71.309186  71.568149  71.840100  72.124892  72.422372  72.732386   
4        71.063352  71.309186  71.568149  71.840100  72.124892  72.422372   
5    

Una primera visualización del metodo Clark-Wright:

In [6]:
from itertools import combinations

In [11]:
clientes_1_data = pd.read_csv('venta_zona_1.csv')
flota_data = pd.read_csv('flota.csv')
camiones_data = pd.read_csv('vehiculos.csv')

# === Calcular demanda por zona ===
demanda_por_zona = clientes_1_data.groupby('id_zona')['venta_digital'].sum().reset_index()
zonas_datos = pd.merge(demanda_por_zona, zonas_data, on='id_zona')

# === Agrupar por tienda física ===
tiendas = zonas_datos['tienda_zona'].unique()
rutas_totales = {}

for tienda in tiendas:
    # Subconjunto de zonas asociadas a esta tienda
    sub_zonas = zonas_datos[zonas_datos['tienda_zona'] == tienda].copy()
    sub_zonas = sub_zonas.reset_index(drop=True)

    # Obtener tipo y cantidad de camiones para la tienda
    flota_info = flota_data[flota_data['id_tienda'] == tienda]
    if flota_info.empty:
        print(f"No hay datos de flota para tienda {tienda}, se omite.")
        continue

    id_camion = flota_info.iloc[0]['id_camion']
    n_camiones = flota_info.iloc[0]['N']
    capacidad = camiones_data.loc[camiones_data['tipo_camion'] == id_camion, 'Q'].values[0]

    # Establecer el depósito como primer punto (usamos la primera zona como proxy del depósito)
    deposito_coord = sub_zonas.iloc[0][['x_zona', 'y_zona']].values
    sub_zonas['nodo'] = range(1, len(sub_zonas) + 1)
    zona_id_map = {row['nodo']: row['id_zona'] for _, row in sub_zonas.iterrows()}

    coords = np.vstack([deposito_coord, sub_zonas[['x_zona', 'y_zona']].values])
    demanda = np.concatenate([[0], sub_zonas['venta_digital'].values])
    dist = cdist(coords, coords)

    rutas = {i: [0, i, 0] for i in range(1, len(coords))}
    savings = []

    for i, j in combinations(range(1, len(coords)), 2):
        s = dist[0, i] + dist[0, j] - dist[i, j]
        savings.append((s, i, j))
    savings.sort(reverse=True)

    for s, i, j in savings:
        ruta_i = next((r for r in rutas.values() if i in r[1:-1]), None)
        ruta_j = next((r for r in rutas.values() if j in r[1:-1]), None)

        if ruta_i is None or ruta_j is None or ruta_i == ruta_j:
            continue

        carga_i = sum(demanda[k] for k in ruta_i if k != 0)
        carga_j = sum(demanda[k] for k in ruta_j if k != 0)
        if carga_i + carga_j > capacidad:
            continue

        if ruta_i[-2] == i and ruta_j[1] == j:
            nueva_ruta = ruta_i[:-1] + ruta_j[1:]
        elif ruta_j[-2] == j and ruta_i[1] == i:
            nueva_ruta = ruta_j[:-1] + ruta_i[1:]
        else:
            continue

        rutas = {k: v for k, v in rutas.items() if v != ruta_i and v != ruta_j}
        rutas[i] = nueva_ruta

    # === Seleccionar como máximo N rutas con menor distancia total ===
    rutas_finales = []
    for ruta in rutas.values():
        zonas_ruta = [zona_id_map[i] if i != 0 else f"DEPOSITO_{tienda}" for i in ruta]
        carga = sum(demanda[i] for i in ruta if i != 0)
        distancia = sum(dist[ruta[k]][ruta[k+1]] for k in range(len(ruta)-1))
        rutas_finales.append({
            'ruta': zonas_ruta,
            'carga': carga,
            'distancia': round(distancia, 2)
        })

    rutas_finales.sort(key=lambda r: r['distancia'])  # priorizar rutas cortas
    rutas_totales[tienda] = rutas_finales[:int(n_camiones)]

# === Mostrar resultados ===
for tienda, rutas in rutas_totales.items():
    print(f"\n Rutas desde tienda: {tienda}")
    for i, r in enumerate(rutas):
        print(f"  Ruta {i+1}: {r['ruta']}, Carga: {r['carga']}, Distancia: {r['distancia']}")


 Rutas desde tienda: 6
  Ruta 1: ['DEPOSITO_6', np.int64(1), np.int64(31), np.int64(61), np.int64(91), np.int64(121), np.int64(151), np.int64(181), np.int64(211), np.int64(241), np.int64(271), np.int64(301), np.int64(331), np.int64(361), np.int64(391), np.int64(421), np.int64(451), np.int64(481), np.int64(511), np.int64(482), np.int64(483), np.int64(484), np.int64(454), np.int64(455), np.int64(456), np.int64(457), np.int64(428), np.int64(427), np.int64(426), np.int64(425), np.int64(424), np.int64(394), np.int64(395), np.int64(396), np.int64(397), np.int64(398), np.int64(369), np.int64(368), np.int64(367), np.int64(366), np.int64(365), np.int64(364), np.int64(334), np.int64(304), np.int64(274), np.int64(244), np.int64(214), np.int64(184), np.int64(154), np.int64(152), np.int64(182), np.int64(212), np.int64(242), np.int64(272), np.int64(302), np.int64(332), np.int64(362), np.int64(392), np.int64(422), np.int64(452), np.int64(453), np.int64(423), np.int64(393), np.int64(363), np.int64(33